# 1. Load Libraries

In [1]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from huggingface_hub import hf_hub_download
import json
import onnxruntime as rt

c:\Users\caleb\OneDrive - National University of Singapore\school\DSA\DSA4264\DSA4264\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
reddit_df = pd.read_csv('../data/combined_cleaned_500k.csv',  lineterminator='\n', encoding='utf8')

In [3]:
reddit_df.head()

,text,timestamp,username,link,link_id,parent_id,id,subreddit_id,moderation\r
0,i think most singaporeans dont give a damn who...,2020-04-11 15:49:23,invigo79,/r/singapore/comments/fz7vtl/im_quite_interest...,t3_fz7vtl,t3_fz7vtl,fn3gbrg,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '..."
1,fair point the secrecy aspect of it slipped my...,2020-04-03 09:59:08,potatetoe_tractor,/r/singapore/comments/fu3axm/government_to_tab...,t3_fu3axm,t1_fmasya5,fmau5k3,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '..."
2,range,2020-02-15 15:07:03,CrossfittJesus,/r/singapore/comments/f4ac70/what_is_ps_defens...,t3_f4ac70,t3_f4ac70,fhp05xc,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '..."
3,gt this is binary thinking because you think t...,2020-06-04 07:07:39,nomad80,/r/singapore/comments/gw55cx/notoracism/fsu4fyd/,t3_gw55cx,t1_fsu3dsf,fsu4fyd,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '..."
4,boo boo poor u lmao,2020-10-31 13:52:12,pirorok,/r/singapore/comments/jl6abo/rsingapore_random...,t3_jl6abo,t1_gap4e9y,gap4vkl,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '..."


# 2. Test on the first 5 rows of data

In [4]:
small_reddit_df = reddit_df[0:5]

In [5]:
small_reddit_df

,text,timestamp,username,link,link_id,parent_id,id,subreddit_id,moderation\r
0,i think most singaporeans dont give a damn who...,2020-04-11 15:49:23,invigo79,/r/singapore/comments/fz7vtl/im_quite_interest...,t3_fz7vtl,t3_fz7vtl,fn3gbrg,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '..."
1,fair point the secrecy aspect of it slipped my...,2020-04-03 09:59:08,potatetoe_tractor,/r/singapore/comments/fu3axm/government_to_tab...,t3_fu3axm,t1_fmasya5,fmau5k3,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '..."
2,range,2020-02-15 15:07:03,CrossfittJesus,/r/singapore/comments/f4ac70/what_is_ps_defens...,t3_f4ac70,t3_f4ac70,fhp05xc,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '..."
3,gt this is binary thinking because you think t...,2020-06-04 07:07:39,nomad80,/r/singapore/comments/gw55cx/notoracism/fsu4fyd/,t3_gw55cx,t1_fsu3dsf,fsu4fyd,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '..."
4,boo boo poor u lmao,2020-10-31 13:52:12,pirorok,/r/singapore/comments/jl6abo/rsingapore_random...,t3_jl6abo,t1_gap4e9y,gap4vkl,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '..."


# 3. Model Text Classification

In [6]:
# Download model config
repo_path = "govtech/lionguard-v1"
config_path = hf_hub_download(repo_id=repo_path, filename="config.json")
with open(config_path, 'r') as f:
    config = json.load(f)

In [7]:
#print(config)

## Embedding Function (Tokenise, Model Configuration and Embeds text data)

In [8]:
def get_embeddings(device, data):
    # Load the model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(config['embedding']['tokenizer'])
    model = AutoModel.from_pretrained(config['embedding']['model'])
    model.eval()
    model.to(device)

    # Generate the embeddings
    batch_size = config['embedding']['batch_size']
    num_batches = int(np.ceil(len(data)/batch_size))
    output = []
    for i in range(num_batches):
        sentences = data[i*batch_size:(i+1)*batch_size]
        encoded_input = tokenizer(sentences, max_length=config['embedding']['max_length'], padding=True, truncation=True, return_tensors='pt')
        encoded_input.to(device)
        with torch.no_grad():
            model_output = model(**encoded_input)
            sentence_embeddings = model_output[0][:, 0]
        sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)
        output.extend(sentence_embeddings.cpu().numpy())
    
    return np.array(output)

## Predict Function (score prediction with and without thresholds)

In [9]:
def predict(batch_text):
    device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
    embeddings = get_embeddings(device, batch_text)
    embeddings_df = pd.DataFrame(embeddings)

    # Prepare input data
    X_input = np.array(embeddings_df, dtype=np.float32)

    # Load the classifiers
    results = {}
    for category, details in config['classifier'].items():
        # Download the classifier from HuggingFace hub
        local_model_fp = hf_hub_download(repo_id=repo_path, filename=config['classifier'][category]['model_fp'])

        # Run the inference
        session = rt.InferenceSession(local_model_fp)
        input_name = session.get_inputs()[0].name
        outputs = session.run(None, {input_name: X_input})

        # If calibrated, return only the prediction for the unsafe class
        if config['classifier'][category]['calibrated']: 
            scores = [output[1] for output in outputs[1]]
        else:
            scores = outputs[1].flatten()
        
        # Generate the predictions depending on the recommended threshold score
        results[category] = {
            'scores': scores,
            'predictions': {
                'high_recall': [1 if score >= config['classifier'][category]['threshold']['high_recall'] else 0 for score in scores],
                'balanced': [1 if score >= config['classifier'][category]['threshold']['balanced'] else 0 for score in scores],
                'high_precision': [1 if score >= config['classifier'][category]['threshold']['high_precision'] else 0 for score in scores]
            }
        }

    return results

## Generate results in another dataframe

In [10]:
# Extract the text data and id from the DataFrame
batch_id = small_reddit_df['id'].tolist()
batch_text = small_reddit_df['text'].tolist()

# Generate the scores and predictions
results = predict(batch_text)

# Prepare results for DataFrame
output_data = []
for i in range(len(batch_text)):
    output_row = {
        'id': batch_id[i],
        'Text': batch_text[i],
    }
    # IMPT! THIS LOOP WILL PRODUCE 32 COLUMNS! COMMENT OUT IF NOT NEEDED!
    for category in results.keys():
        # scores
        output_row[f'{category} Score'] = results[category]['scores'][i]
        # predictions with highest recall
        output_row[f'{category} HR'] = results[category]['predictions']['high_recall'][i]
        # balanced predictions
        output_row[f'{category} B'] = results[category]['predictions']['balanced'][i]
        # predictions with highest precision
        output_row[f'{category} HP'] = results[category]['predictions']['high_precision'][i]
    output_data.append(output_row)

# Create a DataFrame from the results
small_results_df = pd.DataFrame(output_data)

In [ ]:
# Set display option to show all columns
pd.set_option('display.max_columns', None)

# get results table
print(small_results_df)

        id                                               Text  binary Score  \
0  fn3gbrg  i think most singaporeans dont give a damn who...      0.008994   
1  fmau5k3  fair point the secrecy aspect of it slipped my...      0.000000   
2  fhp05xc                                              range      0.004988   
3  fsu4fyd  gt this is binary thinking because you think t...      1.000000   
4  gap4vkl                                boo boo poor u lmao      1.000000   

   binary HR  binary B  binary HP  hateful Score  hateful HR  hateful B  \
0          0         0          0      -0.582897           0          0   
1          0         0          0      -1.116735           0          0   
2          0         0          0      -1.027191           0          0   
3          1         1          1      -0.419287           0          0   
4          1         1          1      -0.952112           0          0   

   hateful HP  harassment Score  harassment HR  harassment B  harassment H

In [12]:
# See column names
print(small_results_df.columns)

Index(['id', 'Text', 'binary Score', 'binary HR', 'binary B', 'binary HP',
       'hateful Score', 'hateful HR', 'hateful B', 'hateful HP',
       'harassment Score', 'harassment HR', 'harassment B', 'harassment HP',
       'public_harm Score', 'public_harm HR', 'public_harm B',
       'public_harm HP', 'self_harm Score', 'self_harm HR', 'self_harm B',
       'self_harm HP', 'sexual Score', 'sexual HR', 'sexual B', 'sexual HP',
       'toxic Score', 'toxic HR', 'toxic B', 'toxic HP', 'violent Score',
       'violent HR', 'violent B', 'violent HP'],
      dtype='object')


## Hateful and Toxic Scores

In [13]:
# get id, hateful and toxic scores only
condensed_small_results_df = small_results_df[['id', 'hateful Score', 'toxic Score']]

In [14]:
condensed_small_results_df

,id,hateful Score,toxic Score
0,fn3gbrg,-0.582897,-0.419336
1,fmau5k3,-1.116735,-1.869360
2,fhp05xc,-1.027191,-0.798018
3,fsu4fyd,-0.419287,1.119167
4,gap4vkl,-0.952112,1.197502


## New dataframe with hateful and toxic scores

In [15]:
# merge the 2 dataframes on 'id'
small_hateful_and_toxic_results_df = pd.merge(small_reddit_df, condensed_small_results_df, on='id', how='inner')

In [16]:
print(small_hateful_and_toxic_results_df)

                                                text            timestamp  \
0  i think most singaporeans dont give a damn who...  2020-04-11 15:49:23   
1  fair point the secrecy aspect of it slipped my...  2020-04-03 09:59:08   
2                                              range  2020-02-15 15:07:03   
3  gt this is binary thinking because you think t...  2020-06-04 07:07:39   
4                                boo boo poor u lmao  2020-10-31 13:52:12   

            username                                               link  \
0           invigo79  /r/singapore/comments/fz7vtl/im_quite_interest...   
1  potatetoe_tractor  /r/singapore/comments/fu3axm/government_to_tab...   
2     CrossfittJesus  /r/singapore/comments/f4ac70/what_is_ps_defens...   
3            nomad80   /r/singapore/comments/gw55cx/notoracism/fsu4fyd/   
4            pirorok  /r/singapore/comments/jl6abo/rsingapore_random...   

     link_id   parent_id       id subreddit_id  \
0  t3_fz7vtl   t3_fz7vtl  fn3gbrg   

# 4. Quick Analysis

In [17]:
# expand resolution to see full text
pd.set_option('display.max_colwidth', None)

small_hateful_and_toxic_results_df[['text', 'hateful Score', 'toxic Score']]

,text,hateful Score,toxic Score
0,i think most singaporeans dont give a damn who taiwan belong to,-0.582897,-0.419336
1,fair point the secrecy aspect of it slipped my mind,-1.116736,-1.869361
2,range,-1.027191,-0.798019
3,gt this is binary thinking because you think that im inherently blind and because of the majority privilege\n\ngt this demonstrates your inability to accept opposing views and have no choice but to resort to using ad hominem by casting me into an opposing the others group making it a black and white binary argument in order to have an attempt in giving supposed strength to your argument\n\nwell this is clearly very complicated for you but to bring this back to where i came in the idea of normalizing edmws tone amp tenor just because you think it is anywhere commendable that they bash anyone outside their approved groups\n\nto which all i said was \n\ngt racist against everyone sounds the worst cesspool of assholes then\n\nany other place this would be a open shut point but and here we are with your spirited defense of this mindset,-0.419287,1.119166
4,boo boo poor u lmao,-0.952112,1.197501


Scores can range is [-1,1], where 1 is positive, -1 is negative in that metric. Overall, hatefulness and toxicity are correlated. However, the last 2 text data say otherwise, Lionguard predicts them as not hateful, but toxic. The last 2 texts also show that the longer text is more hateful, but less toxic than the shorter text.

# 5. Hateful Score + Hateful HR & Toxic Score + Toxic HR Analysis Only (drop other scores)

In [17]:
reddit_df.head()

,text,timestamp,username,link,link_id,parent_id,id,subreddit_id,moderation\r
0,i think most singaporeans dont give a damn who...,2020-04-11 15:49:23,invigo79,/r/singapore/comments/fz7vtl/im_quite_interest...,t3_fz7vtl,t3_fz7vtl,fn3gbrg,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '..."
1,fair point the secrecy aspect of it slipped my...,2020-04-03 09:59:08,potatetoe_tractor,/r/singapore/comments/fu3axm/government_to_tab...,t3_fu3axm,t1_fmasya5,fmau5k3,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '..."
2,range,2020-02-15 15:07:03,CrossfittJesus,/r/singapore/comments/f4ac70/what_is_ps_defens...,t3_f4ac70,t3_f4ac70,fhp05xc,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '..."
3,gt this is binary thinking because you think t...,2020-06-04 07:07:39,nomad80,/r/singapore/comments/gw55cx/notoracism/fsu4fyd/,t3_gw55cx,t1_fsu3dsf,fsu4fyd,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '..."
4,boo boo poor u lmao,2020-10-31 13:52:12,pirorok,/r/singapore/comments/jl6abo/rsingapore_random...,t3_jl6abo,t1_gap4e9y,gap4vkl,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '..."


In [18]:
def predict2(batch_text):
    device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
    embeddings = get_embeddings(device, batch_text)
    embeddings_df = pd.DataFrame(embeddings)

    # Prepare input data
    X_input = np.array(embeddings_df, dtype=np.float32)

    # Define the classifiers we want to focus on
    selected_categories = ['hateful', 'toxic']  # Only focus on 'hateful' and 'toxic'

    # Load the classifiers
    results = {}
    for category in selected_categories:  # Only loop over selected_categories
        # Download the classifier from HuggingFace hub
        local_model_fp = hf_hub_download(repo_id=repo_path, filename=config['classifier'][category]['model_fp'])

        # Run the inference
        session = rt.InferenceSession(local_model_fp)
        input_name = session.get_inputs()[0].name
        outputs = session.run(None, {input_name: X_input})

        # If calibrated, return only the prediction for the unsafe class
        if config['classifier'][category]['calibrated']:
            scores = [output[1] for output in outputs[1]]
        else:
            scores = outputs[1].flatten()

        # Generate the predictions depending on the recommended threshold score
        results[f'{category} Score'] = {  # Directly access 'hateful' and 'toxic' scores
            'scores': scores,
            'predictions': {
                'high_recall': [1 if score >= config['classifier'][category]['threshold']['high_recall'] else 0 for score in scores]
            }
        }
        results[f'{category} HR'] = results[f'{category} Score']['predictions']['high_recall']  # CHANGE 4: Only high_recall predictions

    return results


# 6. Testing on Medium Size Dataset

In [19]:
mid_reddit_df = reddit_df[0:50]

In [20]:
# Ensure text is clean and all entries are strings
mid_reddit_df['text'] = mid_reddit_df['text'].fillna('').astype(str)
batch_text = mid_reddit_df['text'].tolist()
batch_id = mid_reddit_df['id'].tolist()


C:\Users\caleb\AppData\Local\Temp\ipykernel_133204\291247755.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mid_reddit_df['text'] = mid_reddit_df['text'].fillna('').astype(str)


In [21]:
# Generate the scores and predictions
results = predict2(batch_text)

# Prepare results for DataFrame
output_data = []
for i in range(len(batch_text)):
    output_row = {
        'id': batch_id[i],
        'Text': batch_text[i],
    }

    # Directly add 'hateful Score', 'hateful HR', 'toxic Score', and 'toxic HR' to the output
    output_row['hateful Score'] = results['hateful Score']['scores'][i]  
    output_row['hateful HR'] = results['hateful HR'][i]  
    output_row['toxic Score'] = results['toxic Score']['scores'][i]  
    output_row['toxic HR'] = results['toxic HR'][i]  

    output_data.append(output_row)

# Create a DataFrame from the results
mid_results_df = pd.DataFrame(output_data)

In [22]:
# Set display option to show all columns
pd.set_option('display.max_columns', None)

# get results table
print(mid_results_df)

         id                                               Text  hateful Score  \
0   fn3gbrg  i think most singaporeans dont give a damn who...      -0.582897   
1   fmau5k3  fair point the secrecy aspect of it slipped my...      -1.116735   
2   fhp05xc                                              range      -1.027191   
3   fsu4fyd  gt this is binary thinking because you think t...      -0.419287   
4   gap4vkl                                boo boo poor u lmao      -0.952112   
5   g7guz2q  its a very simple trick popular with insta and...      -1.064189   
6   hhwhn2u  because she is angry that despite all the hard...      -0.922030   
7   fmb0nae  sometimes there are restrictions on how often ...      -1.213592   
8   fy05ljb        oo yup thats probably the case im on mobile      -1.126568   
9   h4d6tcs                                  delta counterpart      -1.049220   
10  hcyq9cf                      booster shots on the da house      -0.898221   
11  gsg782b  thankfully i do

In [23]:
# See column names
print(mid_results_df.columns)

Index(['id', 'Text', 'hateful Score', 'hateful HR', 'toxic Score', 'toxic HR'], dtype='object')


In [24]:
# get id, hateful and toxic scores only
condensed_mid_results_df = mid_results_df[['id', 'hateful Score', 'hateful HR',  'toxic Score', 'toxic HR']]

In [25]:
condensed_mid_results_df

,id,hateful Score,hateful HR,toxic Score,toxic HR
0,fn3gbrg,-0.582897,0,-0.419336,0
1,fmau5k3,-1.116735,0,-1.869360,0
2,fhp05xc,-1.027191,0,-0.798018,0
3,fsu4fyd,-0.419287,0,1.119167,1
4,gap4vkl,-0.952112,0,1.197502,1
5,g7guz2q,-1.064189,0,-0.542220,0
6,hhwhn2u,-0.922030,0,0.315609,1
7,fmb0nae,-1.213592,0,-1.785311,0
8,fy05ljb,-1.126568,0,-0.934198,0
9,h4d6tcs,-1.049220,0,-0.892171,0


In [26]:
# merge the 2 dataframes on 'id'
mid_hateful_and_toxic_results_df = pd.merge(mid_reddit_df, condensed_mid_results_df, on='id', how='inner')

In [27]:
mid_hateful_and_toxic_results_df

,text,timestamp,username,link,link_id,parent_id,id,subreddit_id,moderation\r,hateful Score,hateful HR,toxic Score,toxic HR
0,i think most singaporeans dont give a damn who...,2020-04-11 15:49:23,invigo79,/r/singapore/comments/fz7vtl/im_quite_interest...,t3_fz7vtl,t3_fz7vtl,fn3gbrg,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '...",-0.582897,0,-0.419336,0
1,fair point the secrecy aspect of it slipped my...,2020-04-03 09:59:08,potatetoe_tractor,/r/singapore/comments/fu3axm/government_to_tab...,t3_fu3axm,t1_fmasya5,fmau5k3,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '...",-1.116735,0,-1.869360,0
2,range,2020-02-15 15:07:03,CrossfittJesus,/r/singapore/comments/f4ac70/what_is_ps_defens...,t3_f4ac70,t3_f4ac70,fhp05xc,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '...",-1.027191,0,-0.798018,0
3,gt this is binary thinking because you think t...,2020-06-04 07:07:39,nomad80,/r/singapore/comments/gw55cx/notoracism/fsu4fyd/,t3_gw55cx,t1_fsu3dsf,fsu4fyd,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '...",-0.419287,0,1.119167,1
4,boo boo poor u lmao,2020-10-31 13:52:12,pirorok,/r/singapore/comments/jl6abo/rsingapore_random...,t3_jl6abo,t1_gap4e9y,gap4vkl,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '...",-0.952112,0,1.197502,1
5,its a very simple trick popular with insta and...,2020-10-03 00:50:29,FitCranberry,/r/singapore/comments/j3zked/the_magic_a_puddl...,t3_j3zked,t1_g7go6qv,g7guz2q,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '...",-1.064189,0,-0.542220,0
6,because she is angry that despite all the hard...,2021-10-24 20:03:39,StareintotheSun2020,/r/singapore/comments/qeu001/deleted_by_user/h...,t3_qeu001,t1_hhw1usq,hhwhn2u,t5_2qh8c,"{'collapsed_reason_code': None, 'collapsed_rea...",-0.922030,0,0.315609,1
7,sometimes there are restrictions on how often ...,2020-04-03 11:54:04,pm_me_pm_speeches,/r/singapore/comments/fu4ch0/pm_lee_announces_...,t3_fu4ch0,t1_fmaw7ot,fmb0nae,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '...",-1.213592,0,-1.785311,0
8,oo yup thats probably the case im on mobile,2020-07-14 04:04:51,charroxgrin,/r/singapore/comments/hqowfd/rsingapore_random...,t3_hqowfd,t1_fy05ccz,fy05ljb,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, '...",-1.126568,0,-0.934198,0
9,delta counterpart,2021-07-07 15:55:34,SamBellFromSarang,/r/SingaporeRaw/comments/ofac8p/indian_governm...,t3_ofac8p,t1_h4bapip,h4d6tcs,t5_xnx04,"{'collapsed_reason_code': None, 'collapsed_rea...",-1.049220,0,-0.892171,0


# 7a. Scoring for full dataset (part1)

In [29]:
reddit_df.shape
# (396298, 9)
# part 1 is from row 0:198,148
# part 2 is from row 198,149 : 396,297

(396298, 9)

In [30]:
part1_reddit_df = reddit_df[0:10]

In [31]:
# Ensure text is clean and all entries are strings
part1_reddit_df['text'] = part1_reddit_df['text'].fillna('').astype(str)
batch_text_part1 = part1_reddit_df['text'].tolist()
batch_id_part1 =part1_reddit_df['id'].tolist()

/var/folders/cc/b861mgp53v7bmzgrnyrglk0w0000gn/T/ipykernel_59835/4281038490.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  part1_reddit_df['text'] = part1_reddit_df['text'].fillna('').astype(str)


In [32]:
# Generate the scores and predictions
results = predict2(batch_text_part1)

# Prepare results for DataFrame
output_data = []
for i in range(len(batch_text_part1)):
    output_row = {
        'id': batch_id_part1[i],
        'Text': batch_text_part1[i],
    }

    # Directly add 'hateful Score', 'hateful HR', 'toxic Score', and 'toxic HR' to the output
    output_row['hateful Score'] = results['hateful Score']['scores'][i]  
    output_row['hateful HR'] = results['hateful HR'][i]  
    output_row['toxic Score'] = results['toxic Score']['scores'][i]  
    output_row['toxic HR'] = results['toxic HR'][i] 

    output_data.append(output_row)

# Create a DataFrame from the results
part1_results_df = pd.DataFrame(output_data)

In [33]:
# See column names
print(part1_results_df.columns)

Index(['id', 'Text', 'hateful Score', 'hateful HR', 'toxic Score', 'toxic HR'], dtype='object')


In [34]:
# get id, hateful and toxic scores only
condensed_part1_results_df = part1_results_df[['id', 'hateful Score', 'hateful HR',  'toxic Score', 'toxic HR']]

In [35]:
condensed_part1_results_df

,id,hateful Score,hateful HR,toxic Score,toxic HR
0,fn3gbrg,-0.582897,0,-0.419336,0
1,fmau5k3,-1.116736,0,-1.869361,0
2,fhp05xc,-1.027191,0,-0.798019,0
3,fsu4fyd,-0.419287,0,1.119166,1
4,gap4vkl,-0.952112,0,1.197501,1
5,g7guz2q,-1.064189,0,-0.542219,0
6,hhwhn2u,-0.922029,0,0.315610,1
7,fmb0nae,-1.213592,0,-1.785310,0
8,fy05ljb,-1.126568,0,-0.934198,0
9,h4d6tcs,-1.049220,0,-0.892172,0


In [36]:
# merge the 2 dataframes on 'id'
part1_hateful_and_toxic_results_df = pd.merge(part1_reddit_df, condensed_part1_results_df, on='id', how='inner')

In [37]:
part1_hateful_and_toxic_results_df

,text,timestamp,username,link,link_id,parent_id,id,subreddit_id,moderation\r,hateful Score,hateful HR,toxic Score,toxic HR
0,i think most singaporeans dont give a damn who taiwan belong to,2020-04-11 15:49:23,invigo79,/r/singapore/comments/fz7vtl/im_quite_interested_to_know_why_so_many/fn3gbrg/,t3_fz7vtl,t3_fz7vtl,fn3gbrg,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, 'collapsed_reason': None, 'controversiality': 0}\r",-0.582897,0,-0.419336,0
1,fair point the secrecy aspect of it slipped my mind,2020-04-03 09:59:08,potatetoe_tractor,/r/singapore/comments/fu3axm/government_to_table_bill_to_ensure_safe_general/fmau5k3/,t3_fu3axm,t1_fmasya5,fmau5k3,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, 'collapsed_reason': None, 'controversiality': 0}\r",-1.116736,0,-1.869361,0
2,range,2020-02-15 15:07:03,CrossfittJesus,/r/singapore/comments/f4ac70/what_is_ps_defense_imprest/fhp05xc/,t3_f4ac70,t3_f4ac70,fhp05xc,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, 'collapsed_reason': None, 'controversiality': 0}\r",-1.027191,0,-0.798019,0
3,gt this is binary thinking because you think that im inherently blind and because of the majority privilege\n\ngt this demonstrates your inability to accept opposing views and have no choice but to resort to using ad hominem by casting me into an opposing the others group making it a black and white binary argument in order to have an attempt in giving supposed strength to your argument\n\nwell this is clearly very complicated for you but to bring this back to where i came in the idea of normalizing edmws tone amp tenor just because you think it is anywhere commendable that they bash anyone outside their approved groups\n\nto which all i said was \n\ngt racist against everyone sounds the worst cesspool of assholes then\n\nany other place this would be a open shut point but and here we are with your spirited defense of this mindset,2020-06-04 07:07:39,nomad80,/r/singapore/comments/gw55cx/notoracism/fsu4fyd/,t3_gw55cx,t1_fsu3dsf,fsu4fyd,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, 'collapsed_reason': None, 'controversiality': 0}\r",-0.419287,0,1.119166,1
4,boo boo poor u lmao,2020-10-31 13:52:12,pirorok,/r/singapore/comments/jl6abo/rsingapore_random_discussion_and_small_questions/gap4vkl/,t3_jl6abo,t1_gap4e9y,gap4vkl,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, 'collapsed_reason': None, 'controversiality': 0}\r",-0.952112,0,1.197501,1
5,its a very simple trick popular with insta and wedding photogs you just need a surface depth dors not really matter,2020-10-03 00:50:29,FitCranberry,/r/singapore/comments/j3zked/the_magic_a_puddle_of_water_on_the_floor_can_do/g7guz2q/,t3_j3zked,t1_g7go6qv,g7guz2q,t5_2qh8c,"{'removal_reason': None, 'collapsed': False, 'collapsed_reason': None, 'controversiality': 0}\r",-1.064189,0,-0.542219,0
6,because she is angry that despite all the hard times she had stood through with him he seemed to have moved on pretty damn fast from their divorce getting a new gf and dog she feels like he didnt even care about all that she went through and isnt mourning the end of their relationship while she is stuck there\n\ngood news for her is that i read somewhere once that while guys rebound faster long term they take it harder while women take much longer to get through the breakdown of a relationship yet end up happier in the long run\n\nbad news for her is that she does not look good shitting all over female talents just because she cant trust her husband,2021-10-24 20:03:39,StareintotheSun2020,/r/singapore/comments/qeu001/deleted_by_user/hhwhn2u/,t3_qeu001,t1_hhw1usq,hhwhn2u,t5_2qh8c,"{'collapsed_reason_code': None, 'collapsed_reason': None, 'collapsed': False, 'controversiality': 1}\r",-0.922029,0,0.315610,1
7,sometimes there are restrictions on how often people can go out to buy essentials though or how many people per household that isnt the case here,2020-04-03 11:54:04,pm_me_pm_speeches,/r/singapore/comments/fu4ch0/pm_lee_announces_new_stricter_r

In [38]:
# Check number of rows
part1_hateful_and_toxic_results_df.shape

(10, 13)

# 7b Scoring for full dataset (part2)
Duplicate section 7a Scoring for full dataset (part1)



Change all variable names with "part1" to "part2".

Change slicing of dataset: 
part 1 is from row 0:198,148
part 2 is from row 198,149 : 396,297

In [28]:
reddit_df.shape
# (396298, 9)
# part 1 is from row 0:198,148
# part 2 is from row 198,149 : 396,297

(396298, 9)

In [29]:
part2_reddit_df = reddit_df[198149:396297]

In [30]:
# Ensure text is clean and all entries are strings
part2_reddit_df['text'] = part2_reddit_df['text'].fillna('').astype(str)
batch_text_part2 = part2_reddit_df['text'].tolist()
batch_id_part2 =part2_reddit_df['id'].tolist()

C:\Users\caleb\AppData\Local\Temp\ipykernel_133204\451461577.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  part2_reddit_df['text'] = part2_reddit_df['text'].fillna('').astype(str)


In [ ]:
# Generate the scores and predictions
results = predict2(batch_text_part2)

# Prepare results for DataFrame
output_data = []
for i in range(len(batch_text_part2)):
    output_row = {
        'id': batch_id_part2[i],
        'Text': batch_text_part2[i],
    }

    # Directly add 'hateful Score', 'hateful HR', 'toxic Score', and 'toxic HR' to the output
    output_row['hateful Score'] = results['hateful Score']['scores'][i]  
    output_row['hateful HR'] = results['hateful HR'][i]  
    output_row['toxic Score'] = results['toxic Score']['scores'][i]  
    output_row['toxic HR'] = results['toxic HR'][i] 

    output_data.append(output_row)

# Create a DataFrame from the results
part2_results_df = pd.DataFrame(output_data)

In [ ]:
# See column names
print(part2_results_df.columns)

In [ ]:
# get id, hateful and toxic scores only
condensed_part2_results_df = part2_results_df[['id', 'hateful Score', 'hateful HR',  'toxic Score', 'toxic HR']]

In [ ]:
condensed_part2_results_df.head()

In [ ]:
# merge the 2 dataframes on 'id'
part2_hateful_and_toxic_results_df = pd.merge(part2_reddit_df, condensed_part2_results_df, on='id', how='inner')

In [ ]:
part2_hateful_and_toxic_results_df.head()

In [ ]:
# Check number of rows
part2_hateful_and_toxic_results_df.shape